# RNN - hihello / charseq


- 'Hihello' problem
- Data Setting
    - One hot encoding
- Cross entropy loss
- Code run through

### 'hihello' problem
- h i h e l l o 문자열을 예측하는 문제
- we will predict the next character
- how can we represent character in pytorch

### cross entropy loss

카테고리컬한 아웃풋을 예측하는데 많이 쓰는 로스입니다.  
일반적으로 카테고리컬한 아웃풋을 만드는  모델에서는 아웃풋값들을 소프트맥스를 취해서 각 확률로 표현합니다. 그후 정답과 비교하여 out값을 늘리도록 만들어주는것이 cross entorpy loss 의 역할입니다.    

아래 코드로에서 좀더 알아보겠습니다.  

In [3]:
"""
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()

loss = criterion(outpus.view(-1,input_size), Y.view(-1))
# 첫번째 파라미터에서는 모델의 아웃풋을 줘야하고
# 두번째 파라미터에는 모델의 정답 lable 값을 줘야합니다. 
"""

'\n# loss & optimizer setting\ncriterion = torch.nn.CrossEntropyLoss()\n\nloss = criterion(outpus.view(-1,input_size), Y.view(-1))\n# 첫번째 파라미터에서는 모델의 아웃풋을 줘야하고\n# 두번째 파라미터에는 모델의 정답 lable 값을 줘야합니다. \n'

In [33]:
import torch
import torch.nn
import torch.optim as optim
import numpy as np

In [34]:
char_set = ['h','i','e','l','o']
# hyper parameters

input_size = len(char_set)
hidden_size = len(char_set)
learning_rate = 0.1

# data setting
x_data = [[0,1,0,2,3,3]]
x_one_hot = [[1,0,0,0,0],
            [0,1,0,0,0],
            [1,0,0,0,0],
            [0,0,1,0,0],
            [0,0,0,1,0],
            [0,0,0,1,0]]
y_data = [[1,0,2,3,3,4]]

In [35]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

### Code run through (charseq)

In [36]:
sample = "if you want you"

In [37]:
# make dictionary
char_set = list(set(sample))
char_dic = {c:i for i,c in enumerate(char_set)}
print(char_set)
print(char_dic)

['t', 'u', 'i', 'f', 'w', 'o', 'n', 'a', 'y', ' ']
{'t': 0, 'u': 1, 'i': 2, 'f': 3, 'w': 4, 'o': 5, 'n': 6, 'a': 7, 'y': 8, ' ': 9}


In [38]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

In [39]:
# data setting
sample_idx = [char_dic[c] for c in sample]
print(sample_idx)

[2, 3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]


In [40]:
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
# np.eye 는 identitlmatrix 를 만들어주는 함수
# 원핫 인코딩을 만들때 굉장히 유리한것 같다. 
y_data = [sample_idx[1:]]

In [41]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [42]:
x_one_hot

[array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])]

### Code run through

In [46]:
# declare RNN
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True)
# batch_first = first를 하면 batch_dimension이 가장 앞으로 오게 됩니다.
# batch_first gurantees the order of output = (Batch, S, f)

# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

In [48]:
# start training
for i in range(100) :
    optimizer.zero_grad()
    # 이것을 해줘야 새로운 gradient를 구할수 있습니다. 
    # 없으면 기존 gradient에 축적이 되는 형식이 됩니다.
    

    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1,dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    
    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    
    print(i,"loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ",result_str )

0 loss:  2.0641868114471436 prediction:  [[3 0 5 9 6 9 4 8 6 9 6 4 6 5]] true Y:  [[3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]] prediction str:  fto n wyn nwno
1 loss:  1.7525980472564697 prediction:  [[5 9 8 8 1 9 4 8 6 0 9 8 8 1]] true Y:  [[3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]] prediction str:  o yyu wynt yyu
2 loss:  1.5522547960281372 prediction:  [[5 9 8 5 1 9 4 7 5 0 9 8 5 1]] true Y:  [[3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]] prediction str:  o you waot you
3 loss:  1.4330037832260132 prediction:  [[5 9 8 5 1 9 4 7 5 0 9 8 5 1]] true Y:  [[3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]] prediction str:  o you waot you
4 loss:  1.3383539915084839 prediction:  [[5 9 8 5 1 9 4 7 5 0 9 8 5 1]] true Y:  [[3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]] prediction str:  o you waot you
5 loss:  1.2612215280532837 prediction:  [[3 9 8 5 1 9 4 7 5 0 9 8 5 1]] true Y:  [[3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]] prediction str:  f you waot you
6 loss:  1.1957882642745972 prediction:  [[3 9

51 loss:  0.8230361938476562 prediction:  [[3 9 8 5 1 9 4 7 6 0 9 8 5 1]] true Y:  [[3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]] prediction str:  f you want you
52 loss:  0.8224705457687378 prediction:  [[3 9 8 5 1 9 4 7 6 0 9 8 5 1]] true Y:  [[3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]] prediction str:  f you want you
53 loss:  0.8219624161720276 prediction:  [[3 9 8 5 1 9 4 7 6 0 9 8 5 1]] true Y:  [[3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]] prediction str:  f you want you
54 loss:  0.8213645219802856 prediction:  [[3 9 8 5 1 9 4 7 6 0 9 8 5 1]] true Y:  [[3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]] prediction str:  f you want you
55 loss:  0.8208103179931641 prediction:  [[3 9 8 5 1 9 4 7 6 0 9 8 5 1]] true Y:  [[3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]] prediction str:  f you want you
56 loss:  0.8203762769699097 prediction:  [[3 9 8 5 1 9 4 7 6 0 9 8 5 1]] true Y:  [[3, 9, 8, 5, 1, 9, 4, 7, 6, 0, 9, 8, 5, 1]] prediction str:  f you want you
57 loss:  0.8199520707130432 prediction: